In [1]:
from collections import deque
import tkinter as tk
from tkinter import Label, LabelFrame, Button, Entry, OptionMenu, StringVar, Frame,simpledialog, Toplevel
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import re
from tkinter import *
from matplotlib.patches import FancyArrowPatch

In [2]:
class PageReplacementApp(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.master.title("Page Replacement Algorithm")
        self.master.geometry("1200x800")
        self.pack()
        self.create_widgets()
        self.page_frames = []
        self.reference_bits = []
        self.clock_hand = 0
        self.visualization_area = Frame(self)
        self.visualization_area.grid(row=3, columnspan=10, sticky="nsew")

    def create_widgets(self):
        Label(self, text="Page Replacement Algorithm", font=("Century Gothic", 20)).grid(row=0, columnspan=10)
        self.algorithm_choice = StringVar(value="FIFO")
        OptionMenu(self, self.algorithm_choice, "FIFO", "LRU", "Second Chance").grid(row=1, column=0)
        Label(self, text="Number of frames:").grid(row=1, column=1)
        self.no_frames = Entry(self, width=10)
        self.no_frames.grid(row=1, column=2)
        Label(self, text="Page Reference String:").grid(row=1, column=3)
        self.page_ref = Entry(self, width=20)
        self.page_ref.grid(row=1, column=4)
        Button(self, text="Visualize", command=self.visualize).grid(row=1, column=5)
        
    def visualize(self):
        frames = int(self.no_frames.get())
        pages = list(map(int, self.page_ref.get().strip().split()))
        algorithm = self.algorithm_choice.get()
    
        if algorithm == "FIFO":
            results = self.fifo(pages, frames)
        elif algorithm == "LRU":
            results = self.lru(pages, frames)
        #results = self.fifo(pages, frames)
        elif algorithm == "Second Chance":
            results, page_faults = self.second_chance(pages, frames)
        self.update_gui(pages, results)
    
    def fifo(self,pages, capacity):
        frame_list = [None] * capacity
        results = []
        current = 0
        for page in pages:
            if page not in frame_list:
                if None in frame_list:
                    frame_index = frame_list.index(None)
                else:
                    frame_index = current
                    current = (current + 1) % capacity
                frame_list[frame_index] = page
                status = 'Fault'
            else:
                status = 'Hit'
            frame_state = [frame if frame is not None else '-' for frame in frame_list]
            results.append((frame_state, [' '] * capacity, status))
        return results

    def lru(self, pages, num_frames):
        frames = ['-'] * num_frames  # Initialize frames with "-" to indicate empty slots
        last_used = [-1] * num_frames  # Track the last use time of each frame
        page_faults = 0
        hits = 0
        results = []

        for i, page in enumerate(pages):
            if page in frames:
                # Page hit
                hits += 1
                index = frames.index(page)
                last_used[index] = i  # Update last used time
                status = 'Hit'
            else:
                # Page fault
                page_faults += 1
                status = 'Fault'
                if '-' in frames:
                    # There is an empty frame available
                    empty_index = frames.index('-')
                    frames[empty_index] = page
                    last_used[empty_index] = i
                else:
                    # No empty frame, replace the least recently used page
                    lru_index = last_used.index(min(last_used))  # Find index of least recently used
                    frames[lru_index] = page
                    last_used[lru_index] = i

            # Prepare frame state for GUI display
            frame_state = [f if f != '-' else '-' for f in frames]
            results.append((frame_state, [' '] * num_frames, status))

        return results

    def second_chance(self, pages, capacity):
        page_frames = ['-'] * capacity  # Initialize page frames to '-' indicating empty
        reference_bits = [0] * capacity  # Initialize reference bits as 0
        clock_hand = 0
        page_faults = 0
        results = []

        for page in pages:
            if page in page_frames:
                index = page_frames.index(page)
                reference_bits[index] = 1  # Set reference bit to 1 for the accessed page
                results.append((page_frames[:], reference_bits[:], 'Hit'))
            else:
                page_faults += 1
                if len(page_frames) < capacity:
                    page_frames.append(page)
                    reference_bits.append(0)  # Set reference bit to 0 when adding a new page
                else:
                    # Find the oldest page with reference bit 0 (FIFO)
                    victim_index = clock_hand
                    while True:
                        if reference_bits[clock_hand] == 0:
                            victim_index = clock_hand
                            break
                        else:
                            reference_bits[clock_hand]=0
                            clock_hand = (clock_hand + 1) % capacity
                    # Replace the victim page with the new page
                    page_frames[victim_index] = page
                    # Set reference bit to 0 for the replaced page
                    page_frames[victim_index] = page
                    # Set reference bit to 1 for the newly added page
                    #self.reference_bits[self.page_frames.index(page)] = 1

                    # Move the clock hand to the next page after replacement
                    clock_hand = (victim_index + 1) % capacity
                results.append((page_frames[:], reference_bits[:], 'Fault'))

        return results, page_faults

    def update_gui(self, pages, results):
        total_faults = 0
        for widget in self.visualization_area.winfo_children():
            widget.destroy()
        Label(self.visualization_area, text="Reference String", font=("Century Gothic", 12)).grid(row=2, columnspan=len(pages))
        for step, (frames, ref_bits, status) in enumerate(results):
            Label(self.visualization_area, text=str(pages[step]), font=("Century Gothic", 12), width=4).grid(row=3, column=step)
            for i, frame in enumerate(frames):
                label_text = f"{frame}({ref_bits[i]})" if frame != '-' else "-"
                label = Label(self.visualization_area, text=label_text, borderwidth=2, relief="solid", width=6, height=2)
                label.grid(row=i+4, column=step)
            color = "red" if status == 'Fault' else "green"
            status_label = Label(self.visualization_area, text=status, fg=color)
            status_label.grid(row=len(frames)+5, column=step)
            if status == 'Fault':
                total_faults += 1

        Label(self.visualization_area, text=f"Total Page Faults: {total_faults}", font=("Century Gothic", 12)).grid(row=len(frames)+6, columnspan=len(pages))


    


In [3]:
class DiskReplacementApp(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.master.title("Disk Replacement Algorithms")
        self.master.geometry("1200x800")
        self.direction_choice = StringVar(value="right")  # Ensure this is initialized before widgets
        self.algorithm_choice = StringVar(value="SSTF")   # Initialize algorithm choice
        self.create_scrollable_area()

    def create_scrollable_area(self):
        # Create the main canvas and scrollbar
        self.canvas = tk.Canvas(self)
        self.scrollbar = tk.Scrollbar(self, orient="vertical", command=self.canvas.yview)
        self.scrollable_frame = tk.Frame(self.canvas)

        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: self.canvas.configure(
                scrollregion=self.canvas.bbox("all")
            )
        )

        self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        self.canvas.configure(yscrollcommand=self.scrollbar.set)

        self.canvas.pack(side="left", fill="both", expand=True)
        self.scrollbar.pack(side="right", fill="y")

        self.create_widgets()
        self.create_plotting_area()

    def create_widgets(self):
        center_frame = Frame(self.scrollable_frame)
        center_frame.pack(fill="x", expand=True)

        Label(center_frame, text="Disk Scheduling Algorithms", font=("Arial", 16)).grid(row=0, columnspan=2, pady=10)
        Label(center_frame, text="Choose Algorithm:").grid(row=1, column=0, sticky="e")
        OptionMenu(center_frame, self.algorithm_choice, "C-SCAN", "SSTF", "SCAN", command=self.update_direction_option).grid(row=1, column=1, sticky="w")

        # Direction selection (initially hidden)
        self.direction_label = Label(center_frame, text="Choose Direction:")
        self.direction_menu = OptionMenu(center_frame, self.direction_choice, "right", "left")
        self.direction_label.grid(row=2, column=0, sticky="e")
        self.direction_menu.grid(row=2, column=1, sticky="w")
        self.direction_label.grid_remove()
        self.direction_menu.grid_remove()

        Label(center_frame, text="Enter the sequence of disk requests:").grid(row=3, column=0)
        self.disk_requests_entry = Entry(center_frame, width=50)
        self.disk_requests_entry.grid(row=3, column=1)
        Label(center_frame, text="Enter the initial head position:").grid(row=4, column=0)
        self.initial_head_position_entry = Entry(center_frame, width=50)
        self.initial_head_position_entry.grid(row=4, column=1)
        Button(center_frame, text="Visualize", command=self.visualize).grid(row=5, columnspan=2, pady=20)

    def update_direction_option(self, choice):
        if choice == "C-SCAN":
            self.direction_label.grid()
            self.direction_menu.grid()
        else:
            self.direction_label.grid_remove()
            self.direction_menu.grid_remove()

    def create_plotting_area(self):
        self.fig = Figure(figsize=(10, 4), dpi=100)
        self.plot = self.fig.add_subplot(111)
        # Matplotlib Canvas
        self.fig_canvas = FigureCanvasTkAgg(self.fig, master=self.scrollable_frame)
        self.fig_canvas.draw()
        self.fig_canvas.get_tk_widget().pack(fill="both", expand=True)

    def visualize(self):
        requests = self.disk_requests_entry.get()
        initial_position = int(self.initial_head_position_entry.get())
        algorithm = self.algorithm_choice.get()
        direction = self.direction_choice.get()
        
        if requests and initial_position is not None:
            requests = [int(r) for r in re.split(r'\D+', requests) if r]  # Parse requests
            if algorithm == "SCAN":
                total_movement, visited_positions = self.scan(requests, initial_position)
            elif algorithm == "C-SCAN":
                total_movement, visited_positions = self.c_scan(requests, initial_position, 199, direction)  # Example parameters
            elif algorithm == "SSTF":
                total_movement, visited_positions = self.sstf(requests, initial_position)
            self.plot_graph(total_movement, visited_positions, algorithm)

    def scan(self, requests, initial_head_position, disk_size=199):
        # Sorting the disk requests for ordered processing
        requests.sort()
        total_head_movement = 0
        current_position = initial_head_position
        visited_positions = [current_position]

        # Partition requests based on the initial position
        left_requests = [req for req in requests if req < initial_head_position]
        right_requests = [req for req in requests if req >= initial_head_position]

        # Start by moving to the left
        left_requests.sort(reverse=True)  # Sort descending for direct processing

        # Handling left requests
        for request in left_requests:
            total_head_movement += abs(current_position - request)
            visited_positions.append(request)
            current_position = request

        # After servicing left requests, move to 0 if not already there
        if current_position != 0:
            total_head_movement += current_position  # Move to 0
            visited_positions.append(0)
            current_position = 0

        # Now handle the right requests
        for request in right_requests:
            total_head_movement += abs(current_position - request)
            visited_positions.append(request)
            current_position = request

        return total_head_movement, visited_positions

    def c_scan(self, requests, initial_head_position, disk_size, direction):
        total_head_movement = 0
        current_position = initial_head_position
        visited_positions = [initial_head_position]

        if direction == "right":
            requests.sort()
            for request in requests:
                if request >= initial_head_position:
                    total_head_movement += abs(request - current_position)
                    current_position = request
                    visited_positions.append(current_position)
                    if current_position == requests[-1]:
                        visited_positions.append(disk_size)
                        total_head_movement += abs(disk_size - current_position)
                        current_position = 0
                        total_head_movement += abs(disk_size - current_position)
                        visited_positions.append(0)
            for request in requests:
                if request < initial_head_position:
                    total_head_movement += abs(current_position - request)
                    current_position = request
                    visited_positions.append(current_position)
            pass       
            return total_head_movement, visited_positions
        
        elif direction == "left":
            requests.sort(reverse=True)
            for request in requests:
                if request <= initial_head_position:
                    total_head_movement += abs(request - current_position)
                    current_position = request
                    visited_positions.append(current_position)
                    if current_position == requests[-1]:
                        visited_positions.append(0)
                        total_head_movement += abs(current_position - 0)
                        current_position = disk_size
                        total_head_movement += abs(current_position - 0)
                        visited_positions.append(disk_size)
            for request in requests:
                if request > initial_head_position:
                    total_head_movement += abs(current_position - request)
                    current_position = request
                    visited_positions.append(current_position)
            pass        
            return total_head_movement, visited_positions

    def sstf(self, requests, initial_position):
        position = initial_position
        movements = 0
        lis = [position]
        
        while requests:
            nearest = min(requests, key=lambda x: abs(x - position))
            movements += abs(nearest - position)
            position = nearest
            lis.append(position)
            requests.remove(nearest)
        
        return movements, lis
            
    def plot_graph(self, total_movement, positions, title):
        self.plot.clear()  # Clear previous plot
        # Using scatter and plot for better control over markers and lines
        self.plot.scatter(positions, range(len(positions)), color='blue')  # Plot points
        #self.plot.plot(positions, range(len(positions)), linestyle='-', color='blue')  # Plot lines
        for i in range(len(positions) - 1):
            arrow = FancyArrowPatch((positions[i], i), (positions[i+1], i+1),
                                    arrowstyle='->', mutation_scale=20, color='b')
            self.plot.add_patch(arrow)

        self.plot.set_title(f'{title} Disk Scheduling - Total Movements: {total_movement}')
        self.plot.set_xlabel('Disk Position')
        self.plot.set_ylabel('Time (step)')

        # Rotate x-axis labels and set specific tick locations
        self.plot.set_xticks(positions)
        self.plot.set_xticklabels(positions, rotation=45, ha="right")

        # Annotating each point
        for i, txt in enumerate(positions):
            # Adjust text annotations to prevent overlap
            self.plot.annotate(txt, (positions[i], i),
                            textcoords="offset points",  # Offset from the point
                            xytext=(0,10),  # Distance from text to points (x,y)
                            ha='center')  # Horizontal alignment can be left, right or center

        # Highlight total movement on the graph
        self.plot.text(positions[-1], len(positions)-1, f' Total Movement: {total_movement}', 
                    verticalalignment='bottom', horizontalalignment='right',
                    color='red', fontsize=10)

        self.fig_canvas.draw()




In [4]:
def launch_page_replacement():
    new_window = tk.Toplevel()
    app = PageReplacementApp(master=new_window)
    app.pack(fill='both', expand=True)

def launch_disk_replacement():
    new_window = tk.Toplevel()
    app = DiskReplacementApp(master=new_window)
    app.pack(fill='both', expand=True)

In [5]:
def main():
    root = tk.Tk()
    root.title("Main Menu")
    root.geometry("1200x800")

    tk.Label(root, text="Select a Simulation", font=("Arial", 16)).pack(pady=20)

    page_button = tk.Button(root, text="Page Replacements", command=launch_page_replacement)
    page_button.pack(pady=40)

    disk_button = tk.Button(root, text="Disk Replacements", command=launch_disk_replacement)
    disk_button.pack(pady=40)

    root.mainloop()

if __name__ == "__main__":
    main()
